<h1>NBA Shot predictor</h1>

In [2]:
#Set up the library
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [3]:
#Set up the dataframe
df = pd.read_csv('data/NBA Shot Locations 1997 - 2020.csv')
df.head(3)

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,Action Type,...,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Shot Made Flag,Game Date,Home Team,Away Team,Season Type
0,29700427,389,100,Tim Legler,1610612764,Washington Wizards,4,11,22,Jump Shot,...,Right Side(R),8-16 ft.,15,117,109,1,19980102,WAS,IND,Regular Season
1,29700427,406,100,Tim Legler,1610612764,Washington Wizards,4,9,36,Jump Shot,...,Right Side(R),8-16 ft.,14,143,25,0,19980102,WAS,IND,Regular Season
2,29700427,475,100,Tim Legler,1610612764,Washington Wizards,4,3,7,Jump Shot,...,Left Side(L),8-16 ft.,10,-87,55,0,19980102,WAS,IND,Regular Season


Set up a test df to run quick models on. Will use after finishing EDA

In [4]:
#Test Data Frame will be LeBron James, Raptors Player, PG
test_df=df[df['Player Name'] == 'LeBron James']
test_df.head(3)

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,Action Type,...,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Shot Made Flag,Game Date,Home Team,Away Team,Season Type
1194060,20300014,20,2544,LeBron James,1610612739,Cleveland Cavaliers,1,8,56,Jump Shot,...,Right Side(R),8-16 ft.,15,158,0,1,20031029,SAC,CLE,Regular Season
1194061,20300014,28,2544,LeBron James,1610612739,Cleveland Cavaliers,1,7,44,Jump Shot,...,Left Side(L),8-16 ft.,13,-131,2,1,20031029,SAC,CLE,Regular Season
1194062,20300014,35,2544,LeBron James,1610612739,Cleveland Cavaliers,1,7,1,Fadeaway Jump Shot,...,Right Side(R),16-24 ft.,16,163,5,1,20031029,SAC,CLE,Regular Season


## EDA

In [5]:
#Total Amount of Players
df.groupby(['Player ID']).sum()

,Game ID,Game Event ID,Team ID,Period,Minutes Remaining,Seconds Remaining,Shot Distance,X Location,Y Location,Shot Made Flag,Game Date
Player ID,,,,,,,,,,,
3,39242876973,348329,2430414646129,3828,7869,44032,17174,-4630,85324,643,30183858684
7,11932981383,80830,645855713953,964,2571,11528,4425,-10379,27044,166,8011682735
12,208600892,2774,11274289334,26,32,186,67,0,449,2,139931546
15,91008037308,855518,6025302281464,9339,20494,107301,61045,-21843,356813,1621,74854951169
17,34832524061,244133,1815160563615,2671,5505,32913,14233,-27684,90386,473,22514284987
...,...,...,...,...,...,...,...,...,...,...,...
1629744,2430961478,42250,178778016471,312,611,3621,2282,-986,14366,54,2241885448
1629745,109503757,2128,8053063780,14,26,168,71,288,431,1,101001018
1629750,2146243339,40094,157840048324,280,438,2956,760,1053,4646,49,1979128844


2152 players in this data frame 

In [5]:
# Count of each shot
df['Shot Zone Area'].value_counts()

Center(C)                2494850
Left Side(L)              614701
Right Side(R)             565487
Right Side Center(RC)     526949
Left Side Center(LC)      515926
Back Court(BC)             11599
Name: Shot Zone Area, dtype: int64

In [9]:
#As a precentage
df['Shot Zone Area'].value_counts(normalize=True)

Center(C)                0.527507
Left Side(L)             0.129971
Right Side(R)            0.119566
Right Side Center(RC)    0.111417
Left Side Center(LC)     0.109087
Back Court(BC)           0.002452
Name: Shot Zone Area, dtype: float64

More than 50% of the shots come from the center, with left and right side taking 20% almost evenly

** Glossary for the Shot Zone Basic **
* Corner 3: The small area on the left and right side of the court near the baseline. The 3-point line is a few inches closer to the basket in the corners.

* In The Paint: These are shots taken inside the key, the rectangular box close to the basket that measures 15 feet out to the free throw line and 16 feet wide.

* Restricted Area: The four-foot area directly surrounding the basket, marked by a semicircle in the paint.

* Mid-range: Any shot that comes outside the paint but inside the 3-point line.

* Above the Break 3: Any 3-pointer that is not taken from the corners.

* Backcourt: Any shot taken from beyond half court, mostly at the end of quarters to beat the clock.

In [10]:
# Shot zone basic
df['Shot Zone Basic'].value_counts()

Restricted Area          1505921
Mid-Range                1417625
Above the Break 3         828349
In The Paint (Non-RA)     683748
Left Corner 3             147043
Right Corner 3            137365
Backcourt                   9461
Name: Shot Zone Basic, dtype: int64

In [11]:
# SHot zone basic as a percentage
df['Shot Zone Basic'].value_counts(normalize=True)


Restricted Area          0.318409
Mid-Range                0.299740
Above the Break 3        0.175145
In The Paint (Non-RA)    0.144571
Left Corner 3            0.031091
Right Corner 3           0.029044
Backcourt                0.002000
Name: Shot Zone Basic, dtype: float64

More than 30% of the shots came from the Restricted Area

Roll up the classes to make predictions easier

In [8]:
df.columns

Index(['Game ID', 'Game Event ID', 'Player ID', 'Player Name', 'Team ID',
       'Team Name', 'Period', 'Minutes Remaining', 'Seconds Remaining',
       'Action Type', 'Shot Type', 'Shot Zone Basic', 'Shot Zone Area',
       'Shot Zone Range', 'Shot Distance', 'X Location', 'Y Location',
       'Shot Made Flag', 'Game Date', 'Home Team', 'Away Team', 'Season Type'],
      dtype='object')

Lets find out who has taken the most shots

In [16]:
df.groupby('Player Name')['Shot Made Flag'].count().sort_values(ascending=False).head(10)

Player Name
Kobe Bryant        30220
LeBron James       29658
Dirk Nowitzki      26376
Tim Duncan         24221
Vince Carter       22677
Carmelo Anthony    22526
Paul Pierce        21877
Kevin Garnett      21472
Dwyane Wade        20677
Allen Iverson      20284
Name: Shot Made Flag, dtype: int64

The late Kobe Bryant, has taken the most shots, followed by LeBron and Dirk.
Now lets see who has made the most shots

In [8]:
df.groupby('Player Name')['Shot Made Flag'].sum().sort_values(ascending=False).head(10)

Player Name
LeBron James       14881
Kobe Bryant        13534
Dirk Nowitzki      12389
Tim Duncan         12238
Kevin Garnett      10630
Carmelo Anthony    10042
Dwyane Wade         9904
Vince Carter        9849
Paul Pierce         9689
Tony Parker         9390
Name: Shot Made Flag, dtype: int64

Unsuprisingly, the top 3 are the same, however Lebron has made more shots than Kobe, almost 1500 more shots despite taking 300 less. Lets check the averages for the best shooters

In [127]:
#Lets get a df of shots players have  taken split by taken or not
shot=df.groupby(['Player Name','Shot Made Flag']).count()
shot

Game ID  Game Event ID  Player ID  Team ID  \
Player Name        Shot Made Flag                                               
A.C. Green         0                   889            889        889      889   
                   1                   705            705        705      705   
A.J. Bramlett      0                    17             17         17       17   
                   1                     4              4          4        4   
A.J. Guyton        0                   274            274        274      274   
...                                    ...            ...        ...      ...   
Zoran Planinic     1                   197            197        197      197   
Zydrunas Ilgauskas 0                  4994           4994       4994     4994   
                   1                  4534           4534       4534     4534   
Zylan Cheatham     0                     3              3          3        3   
                   1                     2              2          2        2   

                                   Team Name  Period  Minutes Remaining  \
Player Name        Shot Made Flag                                         
A.C. Green         0                     889     889                889   
                   1                     705     705                705   
A.J. Bramlett      0                      17      17                 17   
                   1                       4       4                  4   
A.J. Guyton        0                     274     274                274   
...                                      ...     ...                ...   
Zoran Planinic     1                     197     197                197   
Zydrunas Ilgauskas 0                    4994    4994               4994   
                   1                    4534    4534               4534   
Zylan Cheatham     0                       3       3                  3   
                   1                       2       2                  2   

                                   Seconds Remaining  Action Type  Shot Type  \
Player Name        Shot Made Flag                                              
A.C. Green         0                             889          889        889   
                   1                             705          705        705   
A.J. Bramlett      0                              17           17         17   
                   1                               4            4          4   
A.J. Guyton        0                             274          274        274   
...                                              ...          ...        ...   
Zoran Planinic     1                             197          197        197   
Zydrunas Ilgauskas 0                            4994         4994       4994   
                   1                            4534         4534       4534   
Zylan Cheatham     0                               3            3          3   
                   1                               2            2          2   

                                   Shot Zone Basic  Shot Zone Area  \
Player Name        Shot Made Flag                                    
A.C. Green         0                           889             889   
                   1                           705             705   
A.J. Bramlett      0                            17              17   
                   1                             4               4   
A.J. Guyton        0                           274             274   
...                                            ...             ...   
Zoran Planinic     1                           197             197   
Zydrunas Ilgauskas 0                          4994            4994   
                   1                          4534            4534   
Zylan Cheatham     0                             3               3   
                   1                             2               2   

                                   Shot Zone Range  S

In [128]:
#Only get those numbers
shot['Game ID']

Player Name         Shot Made Flag
A.C. Green          0                  889
                    1                  705
A.J. Bramlett       0                   17
                    1                    4
A.J. Guyton         0                  274
                                      ... 
Zoran Planinic      1                  197
Zydrunas Ilgauskas  0                 4994
                    1                 4534
Zylan Cheatham      0                    3
                    1                    2
Name: Game ID, Length: 4236, dtype: int64

In [133]:
#Lets get shooting percetages
shotpt= df.groupby(['Player Name'])
shotpt


In [134]:
#get it as a percentage
x=shotpt['Shot Made Flag'].value_counts(normalize=True)
x

Player Name         Shot Made Flag
A.C. Green          0                 0.557716
                    1                 0.442284
A.J. Bramlett       0                 0.809524
                    1                 0.190476
A.J. Guyton         0                 0.624146
                                        ...   
Zoran Planinic      1                 0.402863
Zydrunas Ilgauskas  0                 0.524139
                    1                 0.475861
Zylan Cheatham      0                 0.600000
                    1                 0.400000
Name: Shot Made Flag, Length: 4236, dtype: float64

In [135]:
#Add the series into a column
shot['Shooting Pct'] = x
shot

Game ID  Game Event ID  Player ID  Team ID  \
Player Name        Shot Made Flag                                               
A.C. Green         0                   889            889        889      889   
                   1                   705            705        705      705   
A.J. Bramlett      0                    17             17         17       17   
                   1                     4              4          4        4   
A.J. Guyton        0                   274            274        274      274   
...                                    ...            ...        ...      ...   
Zoran Planinic     1                   197            197        197      197   
Zydrunas Ilgauskas 0                  4994           4994       4994     4994   
                   1                  4534           4534       4534     4534   
Zylan Cheatham     0                     3              3          3        3   
                   1                     2              2          2        2   

                                   Team Name  Period  Minutes Remaining  \
Player Name        Shot Made Flag                                         
A.C. Green         0                     889     889                889   
                   1                     705     705                705   
A.J. Bramlett      0                      17      17                 17   
                   1                       4       4                  4   
A.J. Guyton        0                     274     274                274   
...                                      ...     ...                ...   
Zoran Planinic     1                     197     197                197   
Zydrunas Ilgauskas 0                    4994    4994               4994   
                   1                    4534    4534               4534   
Zylan Cheatham     0                       3       3                  3   
                   1                       2       2                  2   

                                   Seconds Remaining  Action Type  Shot Type  \
Player Name        Shot Made Flag                                              
A.C. Green         0                             889          889        889   
                   1                             705          705        705   
A.J. Bramlett      0                              17           17         17   
                   1                               4            4          4   
A.J. Guyton        0                             274          274        274   
...                                              ...          ...        ...   
Zoran Planinic     1                             197          197        197   
Zydrunas Ilgauskas 0                            4994         4994       4994   
                   1                            4534         4534       4534   
Zylan Cheatham     0                               3            3          3   
                   1                               2            2          2   

                                   ...  Shot Zone Area  Shot Zone Range  \
Player Name        Shot Made Flag  ...                                    
A.C. Green         0               ...             889              889   
                   1               ...             705              705   
A.J. Bramlett      0               ...              17               17   
                   1               ...               4                4   
A.J. Guyton        0               ...             274              274   
...                                ...             ...              ...   
Zoran Planinic     1               ...             197              197   
Zydrunas Ilgauskas 0               ...            4994             4994   
                   1               ...            4534             4534   
Zylan Cheatham     0               ...               3                3   
                   1               ...               2          

In [140]:
#Lets Reset the index
shot.reset_index(inplace=True)

In [145]:
#Lets take all the made shots
shot=shot[shot['Shot Made Flag']==1]

In [148]:
#Check
shot

,Player Name,Shot Made Flag,Game ID,Game Event ID,Player ID,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,...,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Game Date,Home Team,Away Team,Season Type,Shooting Pct
1,A.C. Green,1,705,705,705,705,705,705,705,705,...,705,705,705,705,705,705,705,705,705,0.442284
3,A.J. Bramlett,1,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0.190476
5,A.J. Guyton,1,165,165,165,165,165,165,165,165,...,165,165,165,165,165,165,165,165,165,0.375854
7,AJ Hammons,1,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,0.404762
9,AJ Price,1,563,563,563,563,563,563,563,563,...,563,563,563,563,563,563,563,563,563,0.380149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,Zion Williamson,1,172,172,172,172,172,172,172,172,...,172,172,172,172,172,172,172,172,172,0.589041
4229,Zoran Dragic,1,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,0.366667
4231,Zoran Planinic,1,197,197,197,197,197,197,197,197,...,197,197,197,197,197,197,197,197,197,0.402863
4233,Zydrunas Ilgauskas,1,4534,4534,4534,4534,4534,4534,4534,4534,...,4534,4534,4534,4534,4534,4534,4534,4534,4534,0.475861


In [150]:
#There are alot of players with too little shots, lets just take players who have made more than 5000 shots
shot=shot[shot['Game ID']>=5000]

In [157]:
#Lets reduce the columns
shot=shot[['Player Name','Shooting Pct','Game ID']]

In [163]:
#Change the name of Game ID
shot=shot.rename(columns={"Game ID": "Shots Made"})
shot

,Player Name,Shooting Pct,Shots Made
53,Al Harrington,0.441864,5176
55,Al Horford,0.517475,5641
57,Al Jefferson,0.499007,6282
111,Allen Iverson,0.423832,8597
131,Amar'e Stoudemire,0.534766,6422
...,...,...,...
3850,Tim Duncan,0.505264,12238
3917,Tony Parker,0.485799,9390
3933,Tracy McGrady,0.434724,7106
4071,Vince Carter,0.434317,9849


In [164]:
#Lets take a look at the top 10
shot.sort_values(by='Shooting Pct',ascending=False).head(10)

,Player Name,Shooting Pct,Shots Made
1244,Dwight Howard,0.586064,7351
3627,Shaquille O'Neal,0.580606,9173
131,Amar'e Stoudemire,0.534766,6422
55,Al Horford,0.517475,5641
552,Carlos Boozer,0.517338,6430
3189,Pau Gasol,0.507358,8792
3850,Tim Duncan,0.505264,12238
2536,LeBron James,0.501753,14881
1311,Elton Brand,0.500920,6805
57,Al Jefferson,0.499007,6282
